# Initial AI Agent PPO w/o
NN (working) MR2 Drag Disc

In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [2]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import numpy as np
import gymnasium
from time import sleep

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 800
my_config["act_buf_len"] = 4
my_config["reset_act_buf"] = True
my_config["benchmark"] = False
my_config["benchmark_polyak"] = 0.2

my_config["interface_kwargs"] = {
  'debugFlag': False, # do not use render() while True
  'discreteAccel' : True,
  'accelAndBrake' : False,
  'discSteer' : True,
  'contAccelOnly' : False,
  'discAccelOnly' : False,
  'modelMode': 3,
  #  [42, 42, K], [84, 84, K], [10, 10, K], [240, 320, K] and  [480, 640, K]
  'imageWidth' : 84, # there is a default Cov layer for PPO with 240 x 320
  'imageHeight' : 84,
  'trackChoice' : 0, # 0 is HS, 1 is 400m
  'carChoice' : 0, # 0 is MR2, 1 is Supra, 2 is Civic
  'rewardMode' : "simplex"
}

In [5]:
def env_creator(env_config):
    env = gymnasium.make("real-time-gym-v1", config=env_config)
    return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

In [3]:
import ray
ray.shutdown()
ray.init()

2023-10-16 15:52:36,698	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [5]:
from ray.rllib.algorithms.ppo import PPOConfig
config = (
    PPOConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        batch_mode="truncate_episodes",
        #batch_mode="completed_episodes",
        )
    .framework(
        framework="torch",
        )
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
        train_batch_size=400,
        model={
            "custom_model": "GTCustomNet.GTCustomNet",
            "custom_model_config": {
                "fcnet_hiddens":[256,256],
                }
        },
        _enable_learner_api=False
        )
    .rl_module( _enable_rl_module_api=False)
)

In [6]:
# import json

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         try:
#             json.dumps(obj)
#         except TypeError:
#             return "Not serializable object!"

#         return obj

# config_dict = config.to_dict()

# print(json.dumps(config_dict, sort_keys=True, indent=4, cls=CustomEncoder))

In [7]:
algo = config.build()

2023-10-15 16:55:41,733	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=2916) GT Real Time instantiated
(RolloutWorker pid=2916) GT AI Server instantiated for rtgym
(RolloutWorker pid=2916) starting up on localhost port 9999
(RolloutWorker pid=2916) Waiting for a connection


(RolloutWorker pid=2916) 2023-10-15 16:55:48,781	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
2023-10-15 16:55:48,943	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=2916) Connection from ('127.0.0.1', 54851)
(RolloutWorker pid=2916) J:\git\TensorFlowPSX\Py\hsSpaced.csv
(RolloutWorker pid=2916) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=2916) 0 0
(RolloutWorker pid=2916) I found a BOX
(RolloutWorker pid=2916) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=2916) I found a BOX
(RolloutWorker pid=2916) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=2916) I found a BOX
(RolloutWorker pid=2916) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=2916) I found a BOX
(RolloutWorker pid=2916) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=2916) I found Cov2d layers
(RolloutWorker pid=2916) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=2916) I found Cov2d layers
(RolloutWorker pid=2916) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=2916) I found Cov2d layers
(RolloutWorker pid=2916) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=2916) I found a BOX
(RolloutWorker pid=2916) Box([ 0. -

2023-10-15 16:55:51,898	INFO trainable.py:172 -- Trainable.setup took 10.166 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
algo.get_policy().model

GTCustomNet(
  (cnns): ModuleDict(
    (4): VisionNetwork(
      (_convs): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((2, 2, 2, 2))
            (1): Conv2d(1, 16, kernel_size=(8, 8), stride=(4, 4))
          )
        )
        (1): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 2, 1, 2))
            (1): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2))
          )
        )
        (2): SlimConv2d(
          (_model): Sequential(
            (0): Conv2d(32, 256, kernel_size=(11, 11), stride=(1, 1))
          )
        )
        (3): Flatten(start_dim=1, end_dim=-1)
      )
      (_value_branch_separate): Sequential(
        (0): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((2, 2, 2, 2))
            (1): Conv2d(1, 16, kernel_size=(8, 8), stride=(4, 4))
          )
        )
        (1): SlimConv2d(
          (_model): Sequential(
            (0): ZeroPad2d((1, 2, 1, 2))
         

In [9]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=2916) reset triggered
(RolloutWorker pid=2916) Total reward was: 0 after 0 episodes
(RolloutWorker pid=2916) 0 0 64
(RolloutWorker pid=2916) reload save for track : 64
(RolloutWorker pid=2916) reset triggered
(RolloutWorker pid=2916) Total reward was: [0.38575] after 1 episodes
(RolloutWorker pid=2916) 0 0 64
(RolloutWorker pid=2916) reload save for track : 64


(RolloutWorker pid=2916) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.2071256999997182
(RolloutWorker pid=2916)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=2916) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 3.3058197000000007
(RolloutWorker pid=2916)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=2916) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16413150000112608
(RolloutWorker pid=2916)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=2916) reset triggered
(RolloutWorker pid=2916) Total reward was: [0.5605] after 2 episodes
(RolloutWorker pid=2916) 0 0 64
(RolloutWorker pid=2916) reload save for track : 64
Loop:  1


...

(RolloutWorker pid=2916) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 2.9830180000026303
(RolloutWorker pid=2916)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
2023-10-16 00:50:40,566	WARNING worker.py:1986 -- Traceback (most recent call last):
  File "python\ray\_raylet.pyx", line 870, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 921, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GT

IndexError: list index out of range

(RolloutWorker pid=21048) GT Real Time instantiated
(RolloutWorker pid=21048) GT AI Server instantiated for rtgym
(RolloutWorker pid=21048) starting up on localhost port 9999
(RolloutWorker pid=21048) Waiting for a connection
(RolloutWorker pid=21048) Connection from ('127.0.0.1', 56325)
(RolloutWorker pid=21048) J:\git\TensorFlowPSX\Py\hsSpaced.csv
(RolloutWorker pid=21048) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=21048) 0 0
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=21048) I found Cov2d layers
(RolloutWorker pid=21048) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=21048) I found Cov2d 

(RolloutWorker pid=21048) 2023-10-16 00:50:50,647	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=21048) I found Cov2d layers
(RolloutWorker pid=21048) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box([ 0. -1.], 1.0, (2,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box([ 0. -1.], 1.0, (2,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box([ 0. -1.], 1.0, (2,), float64)
(RolloutWorker pid=21048) I found a BOX
(RolloutWorker pid=21048) Box([ 0. -1.], 1.0, (2,), float64)


#Works!

In [6]:
from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-10-15_16-55-415yyb54u3/checkpoint_001221")

(RolloutWorker pid=13152) GT Real Time instantiated
(RolloutWorker pid=13152) GT AI Server instantiated for rtgym
(RolloutWorker pid=13152) starting up on localhost port 9999
(RolloutWorker pid=13152) Waiting for a connection


(RolloutWorker pid=13152) 2023-10-16 15:53:34,260	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=13152) 2023-10-16 15:53:34,354	WARNING catalog.py:625 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-10-16 15:53:34,475	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.models.torch.complex_input_net.ComplexInputNetwork` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=13152) Connection from ('127.0.0.1', 64514)
(RolloutWorker pid=13152) J:\git\TensorFlowPSX\Py\hsSpaced.csv
(RolloutWorker pid=13152) Discrete Accel or Brake (cannot left foot) and Discrete Steering 2
(RolloutWorker pid=13152) 0 0
(RolloutWorker pid=13152) I found a BOX
(RolloutWorker pid=13152) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=13152) I found a BOX
(RolloutWorker pid=13152) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=13152) I found a BOX
(RolloutWorker pid=13152) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=13152) I found a BOX
(RolloutWorker pid=13152) Box(0.0, 1.0, (1,), float64)
(RolloutWorker pid=13152) I found Cov2d layers
(RolloutWorker pid=13152) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=13152) I found Cov2d layers
(RolloutWorker pid=13152) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=13152) I found Cov2d layers
(RolloutWorker pid=13152) Box(0, 255, (84, 84, 1), uint8)
(RolloutWorker pid=13152) I found a BOX
(RolloutWorker p

2023-10-16 15:53:34,588	WARNING catalog.py:625 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!


I found a BOX
Box(0.0, 1.0, (1,), float64)
I found a BOX
Box(0.0, 1.0, (1,), float64)
I found a BOX
Box(0.0, 1.0, (1,), float64)
I found a BOX
Box(0.0, 1.0, (1,), float64)
I found Cov2d layers
Box(0, 255, (84, 84, 1), uint8)
I found Cov2d layers
Box(0, 255, (84, 84, 1), uint8)
I found Cov2d layers
Box(0, 255, (84, 84, 1), uint8)
I found a BOX
Box([ 0. -1.], 1.0, (2,), float64)
I found a BOX
Box([ 0. -1.], 1.0, (2,), float64)
I found a BOX
Box([ 0. -1.], 1.0, (2,), float64)
I found a BOX
Box([ 0. -1.], 1.0, (2,), float64)


2023-10-16 15:53:37,756	INFO trainable.py:172 -- Trainable.setup took 14.213 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [ ]:
result = algo.train() #single try

In [7]:
N = 50000

for n in range(N):
    result = algo.train()
    print("Loop: ", n)
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
        
algo.save()


(RolloutWorker pid=13152) reset triggered
(RolloutWorker pid=13152) Total reward was: 0 after 0 episodes
(RolloutWorker pid=13152) 0 0 64
(RolloutWorker pid=13152) reload save for track : 64


(RolloutWorker pid=13152) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.11700309999991987
(RolloutWorker pid=13152)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  0
Saved 0


(RolloutWorker pid=13152) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 5.903907400000001
(RolloutWorker pid=13152)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=13152) reset triggered
(RolloutWorker pid=13152) Total reward was: [2.27425] after 1 episodes
(RolloutWorker pid=13152) 0 0 64
(RolloutWorker pid=13152) reload save for track : 64


...

(RolloutWorker pid=13152) reset triggered
(RolloutWorker pid=13152) Total reward was: [0.508] after 41 episodes
(RolloutWorker pid=13152) 0 0 64
(RolloutWorker pid=13152) reload save for track : 64


(RolloutWorker pid=13152) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.20699919992739524
(RolloutWorker pid=13152)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Loop:  81


(RolloutWorker pid=13152) c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 5.747671099999479
(RolloutWorker pid=13152)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
2023-10-16 16:29:17,468	ERROR actor_manager.py:507 -- Ray error, taking actor 1 out of service. ray::RolloutWorker.apply() (pid=13152, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000024AEC556160>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anacond

RayTaskError(MemoryError): [36mray::RolloutWorker.apply()[39m (pid=13152, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x0000024AEC556160>)
  File "python\ray\_raylet.pyx", line 877, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 881, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 821, in ray._raylet.execute_task.function_executor
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\_private\function_manager.py", line 670, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\util\tracing\tracing_helper.py", line 460, in _resume_span
    return method(self, *_args, **_kwargs)
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\rllib\utils\actor_manager.py", line 185, in apply
    raise e
  File "c:\Users\nadir\anaconda3\envs\GTRay2.3.0\lib\site-packages\ray\rllib\utils\actor_manager.py",